# Text Preprocessing Lab: Hotel Reviews

## Overview

You've recently joined TextInsight, a customer feedback analysis company that helps businesses understand customer sentiments across various platforms. As a junior data scientist, your first project involves preprocessing a dataset of hotel reviews that will be used to build a sentiment analysis model.

The hotel chain wants to identify common issues mentioned in negative reviews and highlight positive experiences that could be promoted in their marketing. Before any analysis can be performed, the raw text data needs to be cleaned and transformed into a structured format suitable for NLP algorithms.

This lab will guide you through implementing a text preprocessing pipeline following these key steps:

- Text Acquisition and Initial Cleaning
- Tokenization and Structural Decomposition
- Normalization and Standardization
- Noise Filtering and Feature Selection
- Feature Extraction and Representation
- Validation and Iterative Refinement

You'll apply these steps to transform messy, unstructured hotel reviews into clean, analyzable data that can reveal meaningful patterns and insights.

## Step 0: Imports and Data

First, run the provided code cell to import the necessary libraries and download required NLTK resources:

In [ ]:
%conda install nltk

In [ ]:
# Run this cell without changes
import pandas as pd
import numpy as np
import nltk
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Import specific NLTK modules
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist
from nltk import pos_tag
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.util import ngrams

# Download necessary NLTK data
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger_eng')

# Load the hotel reviews dataset
hotel_reviews = pd.read_csv('hotel_reviews.csv')
hotel_reviews.head()

In [ ]:
# Run this cell without changes
# Calculate basic text statistics
hotel_reviews['review_length'] = hotel_reviews['review_text'].apply(len)
hotel_reviews['word_count'] = hotel_reviews['review_text'].apply(lambda x: len(str(x).split()))
hotel_reviews['sentence_count'] = hotel_reviews['review_text'].apply(lambda x: len(sent_tokenize(str(x))))

# Create visualizations to understand the data
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(hotel_reviews['word_count'], bins=20)
plt.title('Distribution of Review Lengths')
plt.xlabel('Word Count')

plt.subplot(1, 2, 2)
sns.boxplot(x='rating', y='word_count', data=hotel_reviews)
plt.title('Review Length by Rating')
plt.xlabel('Rating (1-5)')
plt.ylabel('Word Count')
plt.tight_layout()
plt.show()

## Step 1: Text Cleaning and Tokenization
Implement functions to clean the text and break it down into analyzable units:

In [ ]:
# Tokenize the reviews into sentences and word tokens
hotel_reviews['sentences'] = hotel_reviews['review_text'].apply(sent_tokenize)
hotel_reviews['tokens'] = hotel_reviews['review_text'].apply(word_tokenize)

# Assign the count of unique word tokens across all reviews
lst_tokens = hotel_reviews['tokens']
lst_words = [item for sublist in lst_tokens for item in sublist]

dict_all_tokens = {}
for word in lst_words:
    if word in dict_all_tokens:
        dict_all_tokens[word] += 1
    else:
        dict_all_tokens[word] = 1

all_tokens = [(token, count) for token, count in dict_all_tokens.items()]

unique_token_count_pre = len(all_tokens)

# Create function to clean initial word tokens
def clean_text(text_tokens):
    """
    Clean raw text by:
    1. Converting to lowercase
    2. Removing HTML tags if present
    3. Removing special characters
    4. Handling common abbreviations
    
    Parameters:
    text_tokens: list of word tokens
    
    Returns:
    list: cleaned list of tokens
    """
    # Convert to lowercase
    text_tokens = [text_token.lower() for text_token in text_tokens]
    
    cleaned = []
    # Remove punctuation but keep numbers
    for text_token in text_tokens:    
        no_punc = re.sub(r'[^\w\s]', '', text_token)
    
        if no_punc and not no_punc.isspace():
            cleaned.append(no_punc)

    return cleaned

# Apply cleaning to all reviews
hotel_reviews['clean_tokens'] = hotel_reviews['tokens'].apply(clean_text)

In [ ]:
# Run this cell without changes
# Check results on a sample review
sample_idx = 10  # Review at index 10
print("Original review:")
print(hotel_reviews['review_text'][sample_idx])
print("\nSentences:")
print(hotel_reviews['sentences'][sample_idx])
print("\nTokens (first 20):")
print(hotel_reviews['clean_tokens'][sample_idx][:20])
print("\nUnique Number of Tokens:")
print(unique_token_count_pre)

## Step 2: Stopword Removal and Normalization
Filter out common words that add little analytical value and normalize (lemmatize) the remaining tokens:

In [ ]:
# Get English stopwords as a set
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(tokens):
    """
    Remove stopwords from a list of tokens
    
    Parameters:
    tokens (list): List of word tokens
    
    Returns:
    list: Tokens with stopwords removed
    """
    return [token for token in tokens if token not in stop_words]

# Apply stopword removal
hotel_reviews['filtered_tokens'] = hotel_reviews['clean_tokens'].apply(remove_stopwords)

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Helper function for lemmatization with POS tagging
def get_wordnet_pos(treebank_tag):
    """
    Convert NLTK POS tags to WordNet POS tags
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Lemmatization function
def lemmatize_tokens(tokens):
    """
    Lemmatize tokens with appropriate POS tags
    
    Parameters:
    tokens (list): List of word tokens
    
    Returns:
    list: Lemmatized tokens
    """
    valid_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
    
    # Tag tokens with parts of speech
    tokens_tagged = pos_tag(valid_tokens)
    
    # Convert to WordNet POS tags
    pos_tokens = [(word[0], get_wordnet_pos(word[1])) for word in tokens_tagged]
    
    # Lemmatize with POS tags
    lemmatized = [lemmatizer.lemmatize(word[0], word[1]) for word in pos_tokens]
    
    return lemmatized

# Apply lemmatization
hotel_reviews['lemmatized_tokens'] = hotel_reviews['filtered_tokens'].apply(lemmatize_tokens)

# Calculate stopword reduction percentage
# Assign the count of unique word tokens across all reviews after cleaning and normalizing
all_tokens = []
for tokens in hotel_reviews['lemmatized_tokens']:
    all_tokens.extend(tokens)
unique_token_count_post = len(set(all_tokens))
stopword_reduction_percent = ((unique_token_count_pre - unique_token_count_post) / unique_token_count_pre) * 100

In [ ]:
# Run this cell without changes
# Check results on the sample review
print("Tokens after stopword removal (first 15):")
print(hotel_reviews['filtered_tokens'][sample_idx][:15])
print("\nTokens after lemmatization (first 15):")
print(hotel_reviews['lemmatized_tokens'][sample_idx][:15])
print(f"\nRemoving stopwords and lemmatizing the text has reduced the vocaburlary by {stopword_reduction_percent:.2f}%")

## Step 3: Generate N-grams and Analyze Frequent Terms
Extract word sequences to capture phrases and identify common topics in the reviews:

In [ ]:
# Generate some data
x = np.linspace(0, 10, 100)
y = np.sin(x)

In [ ]:
# Function to generate bigrams and trigrams
def generate_ngrams(tokens, n):
    """
    Generate n-grams from a list of tokens
    
    Parameters:
    tokens (list): List of tokens
    n (int): Size of n-grams to generate
    
    Returns:
    list: List of n-grams as tuples
    """
    result = {}
    for n in n:
        result[f'{n}-grams'] = list(ngrams(tokens, n))
    return result

# Generate bigrams and trigrams - hint - apply lamdba function
b_n = [2]
t_n = [3]

hotel_reviews['bigrams'] = hotel_reviews['lemmatized_tokens'].apply(lambda x: generate_ngrams(x, b_n))
hotel_reviews['trigrams'] = hotel_reviews['lemmatized_tokens'].apply(lambda x: generate_ngrams(x, t_n))

# Combine all lemmatized tokens into a single list for frequency analysis
all_tokens = []
for tokens in hotel_reviews['lemmatized_tokens']:
    all_tokens.extend(tokens)

# Calculate all token frequencies
token_freq = dict(Counter(all_tokens))

# Get all bigrams for frequency analysis
all_bigrams = []
for bigrams in hotel_reviews['bigrams']:
    all_bigrams.extend(bigrams)

# Calculate bigram frequencies
bigram_freq = Counter(all_bigrams)

# Visualize top tokens and bigrams
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
token_freq.plot(20, title="Top 20 Words in Reviews")

plt.subplot(2, 1, 2)
bigram_freq.plot(20, title="Top 20 Bigrams in Reviews")
plt.tight_layout()
plt.show()

# Find top tokens and bigrams for positive and negative reviews
positive_reviews = hotel_reviews[hotel_reviews['rating'] >= 4]
negative_reviews = hotel_reviews[hotel_reviews['rating'] <= 2]

# Collect tokens from positive and negative reviews
positive_tokens = []
for tokens in positive_reviews['lemmatized_tokens']:
    positive_tokens.extend(tokens)

negative_tokens = []
for tokens in negative_reviews['lemmatized_tokens']:
    negative_tokens.extend(tokens)

# Calculate frequencies
positive_freq = None
negative_freq = None

# Visualize top tokens and bigrams
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
positive_freq.plot(20, title="Top 20 Words in Positive Reviews")

plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
negative_freq.plot(20, title="Top 20 Words in Negative Reviews")

## Step 4: Apply Sentiment Analysis to Preprocessed Text
Use the preprocessed text to analyze sentiment and compare with the explicit ratings:

In [ ]:
sia = SentimentIntensityAnalyzer()

# Join lemmatized tokens back into strings for sentiment analysis
hotel_reviews['lemmatized_text'] = hotel_reviews['lemmatized_tokens'].apply(lambda x: ' '.join(x))

# Calculate sentiment scores
hotel_reviews['sentiment_scores'] = hotel_reviews['lemmatized_text'].apply(lambda x: sia.polarity_scores(x))
hotel_reviews['sentiment_compound'] = hotel_reviews['sentiment_scores'].apply(lambda x: x['compound'])

# Classify sentiment based on compound score
def classify_sentiment(score):
    """
    Classify sentiment based on compound score
    
    Parameters:
    score (float): Compound sentiment score
    
    Returns:
    str: Sentiment classification (positive, negative, or neutral)
    """
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

hotel_reviews['predicted_sentiment'] = hotel_reviews['sentiment_compound'].apply(classify_sentiment)


# Create a new column that compares the predicted sentiment with the actual rating
def compare_sentiment_with_rating(row):
    """
    Compare predicted sentiment with rating
    
    Parameters:
    row: DataFrame row
    
    Returns:
    str: Match status (match or mismatch)
    """
    # Convert rating to sentiment category
    actual_sentiment = 'positive' if row['rating'] >= 4 else ('negative' if row['rating'] <= 2 else 'neutral')
    
    # Compare with predicted sentiment - find where they match/equal
    if hotel_reviews['predicted_sentiment'] == hotel_reviews['rating'].apply(lambda x: actual_sentiment(x)):
        return 'match'
    else:
        return 'mismatch'

hotel_reviews['sentiment_match'] = hotel_reviews.apply(compare_sentiment_with_rating, axis=1) 

# Calculate sentiment accuracy
sentiment_accuracy = (hotel_reviews['sentiment_match'] == 'match').mean() * 100

# Analyze mismatches
mismatches = hotel_reviews['sentiment_match' == 'mismatch']
mismatches_by_rating = mismatches['rating'].value_counts().sort_index()
mismatches_by_rating

## Step 5: Evaluate the Preprocessing Pipeline
Analyze how your preprocessing choices affected the results:

In [ ]:
# Create a function to measure vocabulary size at each step
def calculate_vocabulary_sizes(df):
    """
    Calculate vocabulary size at each preprocessing step
    
    Parameters:
    df (DataFrame): DataFrame with tokens at different preprocessing stages
    
    Returns:
    dict: Vocabulary sizes for each step
    """
    # Original tokens
    original_vocab = set()
    col_name1 = 'tokens'
    for item in df[col_name1]:
        original_vocab.add(item)

    # After stopword removal
    col_name2 = 'filtered_tokens'
    filtered_vocab = set()
    for item in df[col_name2]:
        filtered_vocab.add(item)
    
    # After lemmatization
    col_name3 = 'lemmatized_tokens'
    lemmatized_vocab = set()
    for item in df[col_name3]:
        lemmatized_vocab.add(item)
    
    return {
        'original': len(original_vocab),
        'filtered': len(filtered_vocab),
        'lemmatized': len(lemmatized_vocab)
    }

vocab_sizes = calculate_vocabulary_sizes(hotel_reviews)

# Calculate token reduction percentages
original_to_filtered = (1 - len(filtered_vocab) / len(original_vocab)) * 100
filtered_to_lemmatized = (1 - len(lemmatized_vocab) / len(filtered_vocab)) * 100
total_reduction = (1 - len(lemmatized_vocab) / len(original_vocab)) * 100

In [ ]:
hotel_reviews.head()

In [ ]:
orig_tokens = set(hotel_reviews['tokens'])
orig_tokens

In [ ]:
# Run this cell without changes
# Summarize vocabulary size changes
print(f"Original vocabulary size: {vocab_sizes['original']} unique tokens")
print(f"After stopword removal: {vocab_sizes['filtered']} unique tokens ({original_to_filtered:.1f}% reduction)")
print(f"After lemmatization: {vocab_sizes['lemmatized']} unique tokens ({filtered_to_lemmatized:.1f}% further reduction)")
print(f"Total vocabulary reduction: {total_reduction:.1f}%")

# Create a bar chart showing vocabulary size reduction
plt.figure(figsize=(10, 6))
plt.bar(['Original', 'After Stopwords', 'After Lemmatization'], 
        [vocab_sizes['original'], vocab_sizes['filtered'], vocab_sizes['lemmatized']])
plt.title('Vocabulary Size Reduction through Preprocessing')
plt.ylabel('Number of Unique Tokens')
plt.show()